In [1]:
import numpy as np
import torch 
import sys
sys.path.append("../")
import scipy

import pandas as pd
import matplotlib.pyplot as plt
# import utils_BayTIDE as utils
from scipy.io import loadmat

In [3]:
# impoet tensor.mat using scipy
tensor = loadmat('/data/fang/BayTIDE_data/raw/guangzhou/Data/tensor.mat')
tensor = tensor['tensor']
print(tensor.shape)

# only use the first 5 days, resahpe to 2D
# sub_tensor = tensor[:,:5,:] 
sub_tensor = tensor.reshape(214,61*144)
print(sub_tensor.shape)

# to match the TIDER, just use the first 500 time points
sub_tensor = sub_tensor[:,-501:-1]
print(sub_tensor.shape)

(214, 61, 144)
(214, 8784)
(214, 500)


In [5]:
# check the zero ratio
print(np.sum(sub_tensor==0)/sub_tensor.size)

# print max, min and mean
print(np.max(sub_tensor))
print(np.min(sub_tensor))
print(np.mean(sub_tensor))

0.0
73.195
1.484
37.43098849065421


In [7]:
data_save = {}
data_save['ndims'] = sub_tensor.shape

data_save['raw_data'] = sub_tensor

data_save['data'] = []

# sorted unique timestamps, set as (normalized to 0-1 ) regular time intervals at current stage
data_save['time_uni'] = np.linspace(0,1,sub_tensor.shape[1])

def generate_random_mask( shape, drop_rate=0.2, valid_rate=0.1):
    """
    train_ratio: 1-valid_rate-drop_rate
    test_ratio: drop_rate
    valid_ratio: valid_rate
    """
    N,T = shape

    train_T = int(T*(1-valid_rate-drop_rate))

    T_index = np.arange(T)

    np.random.shuffle(T_index)

    mask_train = np.zeros((N,T))
    mask_test = np.zeros((N,T))
    mask_valid = np.zeros((N,T))

    mask_train[:,T_index[:train_T]] = 1
    mask_test[:,T_index[train_T:train_T+int(T*drop_rate)]] = 1
    mask_valid[:,T_index[train_T+int(T*drop_rate):]] = 1

    # mask_train_list = []
    # mask_test_list = []
    # mask_valid_list = []

    # make block-wise missingness


    # for t in range(T):

    #     mask = np.random.rand(N)
    #     mask_train = np.where(mask>drop_rate+valid_rate, 1, 0)
    #     mask_test = np.where(mask<drop_rate, 1, 0)
    #     mask_valid = np.where((mask>drop_rate) & (mask<drop_rate+valid_rate), 1, 0)

    #     mask_train_list.append(mask_train)
    #     mask_test_list.append(mask_test)
    #     mask_valid_list.append(mask_valid)
    
    # mask_train = np.stack(mask_train_list, axis=1)
    # mask_test = np.stack(mask_test_list, axis=1)
    # mask_valid = np.stack(mask_valid_list, axis=1)

    return mask_train, mask_test, mask_valid

fold = 2
drop_rate = 0.2
valid_rate = 0.1

for i in range(fold):
    mask_train, mask_test, mask_valid = generate_random_mask(sub_tensor.shape, drop_rate, valid_rate)
    data_save['data'].append({'mask_train':mask_train, 'mask_test':mask_test, 'mask_valid':mask_valid})

file_name = '/data/fang/BayTIDE_data/guangzhou_impute'+'_r_%.1f'%(drop_rate)+'_irregulate.npy'
np.save(file_name, data_save)


In [6]:
mask_test.sum()/mask_train.size

0.20118691588785045

In [8]:
file_name

'/data/fang/BayTIDE_data/guangzhou_impute_r_0.2_irregulate.npy'